#### Importing the necessary libraries

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)# To display all the columns
import seaborn as sns
import matplotlib.pyplot as plt

C:\Users\Sannan Dabir\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


#### Reading the dataframe

In [2]:
data=pd.read_csv('spykar_3.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581176 entries, 0 to 581175
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             581176 non-null  int64  
 1   placed_date            581176 non-null  object 
 2   store_id               581176 non-null  int64  
 3   item_code              581176 non-null  object 
 4   item_id                581176 non-null  int64  
 5   l1_category            581176 non-null  object 
 6   l0_category            554449 non-null  object 
 7   l2_category            554449 non-null  object 
 8   mrp                    581176 non-null  float64
 9   disc_percent           581176 non-null  float64
 10  season                 545619 non-null  object 
 11  qty                    581176 non-null  int64  
 12  attributes             581176 non-null  object 
 13  item_colour            575331 non-null  object 
 14  item_pattern           462785 non-nu

#### Data Cleaning and Preparation

In [4]:
def preprocess_dataframe(input_df):
    """
    Preprocesses the input DataFrame according to the specified operations.

    Args:
    - input_df (pd.DataFrame): The input DataFrame to be processed.

    Returns:
    - pd.DataFrame: The processed DataFrame.
    """
    # Convert 'placed_date' column to datetime
    input_df['placed_date'] = pd.to_datetime(input_df['placed_date'])

    # Drop the 'Unnamed: 0' column
    input_df.drop('Unnamed: 0', axis=1, inplace=True)

    # Replace 'Men,Women' with 'Unisex' in the 'item_gender' column
    input_df['item_gender'] = input_df['item_gender'].replace({'Men,Women': 'Unisex'})

    # Replace specified values in the 'season' column
    input_df['season'] = input_df['season'].replace({'CORE': 'Core', 'AllSeason': 'All Season', 'Winter 2022': 'Winter-2022', 'Winter - 2021': 'Winter-2021'})

    # Replace specified values in the 'item_occasion' column
    input_df['item_occasion'] = input_df['item_occasion'].replace({'Daily wear': 'Daily Wear', 'Dailywear': 'Daily Wear', 'CASUAL': 'Casual'})

    # Remove rows where 'l0_category' is in the specified list
    input_df = input_df[~((input_df['l0_category'] == 'Personal Care, Personal Care') | 
                          (input_df['l0_category'] == 'Accessories, Accessories') | 
                          (input_df['l0_category'] == 'Clothing, Clothing'))]

    # Return the processed DataFrame
    return input_df

In [5]:
# Assuming your original DataFrame is named 'original_data'
data = preprocess_dataframe(data)

##### Cleaning the item colour column as it has large number of repeated and incorrectly labelled values

In [6]:
def preprocess_colors(input_df):
    """
    Preprocesses color-related columns in the input DataFrame according to the specified operations.

    Args:
    - input_df (pd.DataFrame): The input DataFrame to be processed.

    Returns:
    - pd.DataFrame: The processed DataFrame.
    """
    # Function to replace 'item_colour' with 'item_primary_color' if 'item_colour' contains a number
    def replace_color(row):
        if pd.notna(row['item_colour']) and any(char.isdigit() for char in str(row['item_colour'])):
            return row['item_primary_color']
        else:
            return row['item_colour']

    # Apply the function to create a new column 'new_item_colour'
    input_df['new_item_colour'] = input_df.apply(replace_color, axis=1)

    # Drop the original 'item_colour' column and rename 'new_item_colour' to 'item_colour'
    input_df = input_df.drop('item_colour', axis=1).rename(columns={'new_item_colour': 'item_colour'})

    # Converting the color labels into title format
    input_df['item_colour'] = input_df['item_colour'].str.title()

    # Replacing '-' and '_' with space in 'item_colour' values
    input_df['item_colour'] = input_df['item_colour'].str.replace('[-_]', ' ', regex=True)

    # Replacing '/' with ' & ' in the 'updated_item_colour' column
    input_df['item_colour'] = input_df['item_colour'].str.replace('/', ' & ')

    # Replace 'Lt.' with 'Light'
    input_df['item_colour'] = input_df['item_colour'].str.replace('Lt.', 'Light ')

    # Replace 'Dk.' with 'Dark'
    input_df['item_colour'] = input_df['item_colour'].str.replace('Dk.', 'Dark ')

    # Replace 'Dk.' with 'Dark'
    input_df['item_colour'] = input_df['item_colour'].str.replace('Dk ', 'Dark')

    # Replace extra spaces with a single space
    input_df['item_colour'] = input_df['item_colour'].str.replace(r'\s+', ' ')

    # Custom function to add 'Blue' in front of 'Navy' where there is no 'Navy Blue' and 'Blue' is not already present
    def add_blue_to_navy(color):
        if pd.notna(color) and 'Navy' in color and 'Navy Blue' not in color and 'Blue' not in color:
            return color.replace('Navy', 'Navy Blue')
        return color

    # Apply the custom function to create a new column 'updated_item_colour'
    input_df['item_colour'] = input_df['item_colour'].apply(add_blue_to_navy)

    # Replacing the same pair of colors with a single value
    input_df['item_colour'] = input_df['item_colour'].replace({'Pink & White': 'White & Pink',
                                                   'Red & White': 'White & Red',
                                                   'White & Orange': 'Orange & White',
                                                   'Grey & White': 'White & Grey',
                                                   'White & Green': 'Green & White',
                                                   'White & Yellow': 'Yellow & White',
                                                   'Grey & Blue': 'Blue & Grey',
                                                   'Grey & Yellow': 'Yellow & Grey',
                                                   'Blue & Green': 'Green & Blue',
                                                   'Blue & Pink': 'Pink & Blue',
                                                   'Mid_Blue': 'Mid Blue',
                                                   'Blue & Brown': 'Brown & Blue',
                                                   'Peach & White': 'White & Peach',
                                                   'White & Khaki': 'Khaki & White',
                                                   'Blue & White': 'White & Blue',
                                                   'White & Navy': 'Navy & White',
                                                   'Navy & Khaki': 'Khaki & Navy',
                                                   'Blue & Brown': 'Brown & Blue',
                                                   'Blue & Yellow': 'Yellow & Blue',
                                                   'Blue & Red': 'Red & Blue',
                                                   'Khaki & White': 'White & Khaki',
                                                   'Orange & White': 'White & Orange',
                                                   'White & Red': 'Red & White',
                                                   'Black & Blue': 'Blue & Black',
                                                   'White & Black': 'Black & White',
                                                   'Red & Blue': 'Blue & Red',
                                                   'Grey & Olive Green': 'Olive Green & Grey',
                                                   'Green & Navy Blue': 'Navy Blue & Green',
                                                   'Grey & Navy Blue': 'Navy Blue & Grey',
                                                   'Navy Blue & Black & White': 'Black & White & Navy Blue',
                                                   'Grey Melange & Black': 'Black & Grey Melange',
                                                   'Grey & Melange': 'Melange & Grey',
                                                   'Black & Grey Melange': 'Grey Melange & Black',
                                                   'Navy Blue & Grey Melange': 'Grey Melange & Navy Blue',
                                                   'Anthra Melange & Yellow': 'Yellow & Anthra Melange',
                                                   'White & Grey Melange': 'Grey Melange & White',
                                                   'White & Navy Blue': 'Navy Blue & White',
                                                   'Wine & Navy Blue': 'Navy Blue & Wine',
                                                   'Olive Green & White': 'White & Olive Green',
                                                   'Grey & Maroon': 'Maroon & Grey',
                                                   'Grey Melange & Navy Blue': 'Navy Blue & Grey Melange',
                                                   'White & Navy Blue & Red': 'Navy Blue & Red & White',
                                                   'Red & Navy Blue': 'Navy Blue & Red'
                                                               })

    return input_df

In [7]:
data = preprocess_colors(data)

#### Checking the number of nan values in each columns

In [8]:
def preprocess_fillna_conflict(data):
    """
    Preprocesses missing values, fills NaN values with mode within each 'item_id' group,
    and removes rows with conflicting values within specified columns.

    Args:
    - data (pd.DataFrame): The input DataFrame to be processed.

    Returns:
    - pd.DataFrame: The processed DataFrame.
    """
    def fillna_mode(series):
        mode_values = series.mode()
        if not mode_values.empty:
            return series.fillna(mode_values.iloc[0])
        else:
            return series

    # Apply the fillna_mode function to each group defined by 'item_id'
    df_filled = data.groupby('item_id').transform(fillna_mode)

    # If you have specific columns to apply this operation, you can use:
    # df_filled = df.groupby('item_id')[your_columns].transform(fillna_mode)

    # Replace the original DataFrame with the filled one
    data.update(df_filled)

    columns_to_check = ['l1_category', 'l0_category', 'l2_category', 'season', 'item_colour',
                        'item_pattern', 'item_primary_color', 'item_primary_material',
                        'item_product_fit', 'item_occasion', 'item_product_details',
                        'item_gender', 'item_neck_type', 'item_sleeve_length', 'item_topwear_length']

    # Group by 'item_id' and check the number of unique values in each specified column
    unique_combinations = data.groupby('item_id')[columns_to_check].nunique()

    # Identify item IDs with conflicting values in any of the specified columns
    conflicting_ids = unique_combinations[unique_combinations > 1].dropna(how='all')

    # Store conflicting item IDs in a list
    conflicting_ids_list = list(conflicting_ids.index)

    # Removing the conflicting item ids from the main data
    data = data[~(data['item_id'].isin(conflicting_ids_list))]

    return data

In [9]:
data = preprocess_fillna_conflict(data)

C:\Users\Sannan Dabir\AppData\Local\Temp\ipykernel_28476\1762651946.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return series.fillna(mode_values.iloc[0])


#### Creating a main dataframe with all the required columns

In [10]:
def preprocess_merge_and_group(input_df):
    """
    Preprocesses the input DataFrame by grouping and merging based on specified columns.

    Args:
    - input_df (pd.DataFrame): The input DataFrame to be processed.

    Returns:
    - pd.DataFrame: The processed DataFrame.
    """
    # Group by specified columns and sum the 'qty' column
    grouped_df = input_df.groupby(['placed_date', 'store_id', 'item_id', 'mrp'])['qty'].sum().reset_index()

    # Merge with selected columns from the original DataFrame
    main_df = pd.merge(grouped_df,
                       input_df[['item_id', 'item_code', 'l1_category', 'l0_category', 'l2_category',
                                  'season', 'item_colour', 'item_pattern', 'item_primary_color',
                                  'item_primary_material', 'item_product_fit', 'item_occasion',
                                  'item_gender', 'item_neck_type', 'item_sleeve_length',
                                  'item_topwear_length', 'item_brand_name', 'item_age_group']],
                       on='item_id',
                       how='left')

    # Drop duplicates based on all columns
    main_df = main_df.drop_duplicates()

    return main_df

In [11]:
# Assuming 'data' is your original DataFrame
main_df = preprocess_merge_and_group(data)

#### Adding new feature to find initial introduction days and days since introduction of a product

In [12]:
def preprocess_dates_and_mrp(input_df):
    """
    Preprocesses the input DataFrame by calculating initial introduction date,
    days since introduction, and initial MRP for each item.

    Args:
    - input_df (pd.DataFrame): The input DataFrame to be processed.

    Returns:
    - pd.DataFrame: The processed DataFrame.
    """
    # Find the minimum placed date per item
    min_date_per_item = input_df.groupby('item_id')['placed_date'].min().reset_index()
    min_date_per_item.columns = ['item_id', 'initial_introduction_date']

    # Merge with the main DataFrame
    main_df = pd.merge(input_df, min_date_per_item, on='item_id', how='left')

    # Calculate days since introduction
    main_df['days_since_introduction'] = (main_df['placed_date'] - main_df['initial_introduction_date']).dt.days

    # Sort the DataFrame by 'item_id' and 'placed_date' in ascending order
    main_df.sort_values(['item_id', 'placed_date'], inplace=True)

    # Add a new column 'initial_mrp' capturing the first MRP for each item
    main_df['initial_mrp'] = main_df.groupby('item_id')['mrp'].transform('first')

    return main_df

In [13]:
main_df = preprocess_dates_and_mrp(main_df)

In [14]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 567675 entries, 176810 to 559559
Data columns (total 25 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   placed_date                567675 non-null  datetime64[ns]
 1   store_id                   567675 non-null  int64         
 2   item_id                    567675 non-null  int64         
 3   mrp                        567675 non-null  float64       
 4   qty                        567675 non-null  int64         
 5   item_code                  567675 non-null  object        
 6   l1_category                567675 non-null  object        
 7   l0_category                566866 non-null  object        
 8   l2_category                566866 non-null  object        
 9   season                     533819 non-null  object        
 10  item_colour                562314 non-null  object        
 11  item_pattern               455599 non-null  object  

#### Exporting the main_df to a csv file

In [44]:
main_df.to_csv('Main_Working_Data.csv', index=False)